# Parquet, DuckDB, and Arrow: Columnar data examples

Let's start by accessing an existing 1.3 GB parquet file over the web.

In [ ]:
import duckdb
import os

['stat', 'bounds', 'min', 'max', 'mean', 'median', 'std', 'var', 'sum', 'path', 'tile', 'z']


In [12]:
iwp_path = 'https://arcticdata.io/data/10.18739/A24F1MK7Q/iwp_geotiff_low_medium/raster_summary.parquet'
iwp = duckdb.read_parquet(iwp_path)
print(iwp.columns)

['stat', 'bounds', 'min', 'max', 'mean', 'median', 'std', 'var', 'sum', 'path', 'tile', 'z']


In [2]:
duckdb.sql("SELECT count(*) as n FROM iwp;").show()

┌──────────┐
│    n     │
│  int64   │
├──────────┤
│ 18150329 │
└──────────┘



In [3]:
duckdb.sql("select distinct stat from iwp order by stat;").show()

┌──────────────┐
│     stat     │
│   varchar    │
├──────────────┤
│ iwp_coverage │
└──────────────┘



In [5]:
low_coverage = iwp.project("bounds, sum").filter("sum < 10")
low_coverage.count('*')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        37169 │
└──────────────┘

In [9]:
low_coverage.write_parquet("low_coverage.parquet")
os.stat("low_coverage.parquet").st_size / (1024 * 1024)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

1.4038171768188477

In [11]:
duckdb.sql("SELECT * from low_coverage order by sum desc").limit(10).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────────────────────────────────────┬───────────────────┐
│                                      bounds                                      │        sum        │
│                                     varchar                                      │      double       │
├──────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│ [116.99340820312375, 117.00439453124875, 71.32324218750009, 71.33422851562509]   │ 9.999709300915525 │
│ [116.99340820312375, 117.00439453124875, 71.32324218750009, 71.33422851562509]   │ 9.999709300915525 │
│ [-77.78320312500044, -77.76123046875044, 81.47460937500003, 81.49658203125003]   │ 9.999367985118123 │
│ [-154.6655273437501, -154.6435546875001, 70.88378906250009, 70.90576171875009]   │ 9.998618022818269 │
│ [-116.36169433593756, -116.35620117187506, 77.39868164062501, 77.40417480468751] │ 9.998248625671277 │
│ [95.05920410156227, 95.06469726562477, 71.65832519531

## Delta Fisheries using Arrow

In this example, we'll read in a dataset of fish abundance in the San Francisco Estuary, which is published in csv format on the [Environmental Data Initiative](https://portal.edirepository.org/nis/mapbrowse?scope=edi&identifier=1075&revision=1). This dataset isn't huge, but it is big enough (3 GB) that working with it locally can be fairly taxing on memory. Motivated by user difficulties in actually working with the data, the [`deltafish` R](https://github.com/Delta-Stewardship-Council/deltafish) package was written using the R implementation of `arrow`. It works by downloading the EDI repository data, writing it to a local cache in parquet format, and using `arrow` to query it. In this example, I've put the Parquet files in a sharable location so we can explore them using `pyarrow`.


In [13]:
import pyarrow.dataset as ds
import numpy as np
import pandas as pd

In [14]:
deltafish = ds.dataset("/home/shares/deltafish/fish",
                       format="parquet",
                       partitioning='hive')

In [15]:
deltafish.files

['/home/shares/deltafish/fish/Taxa=Acanthogobius flavimanus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Acipenser medirostris/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Acipenser transmontanus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Acipenser/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Actinopteri/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Aequorea/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Allosmerus elongatus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Alopias vulpinus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Alosa sapidissima/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Alosinae/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Ameiurus catus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Ameiurus melas/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Ameiurus natalis/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Ameiurus nebulosus/part-0.parquet',
 '/home/shares/deltafish/fish/Taxa=Ammo

In [16]:
deltafish.schema

SampleID: string
Length: double
Count: double
Notes_catch: string
Taxa: string

In [17]:
expr = ((ds.field("Taxa")=="Dorosoma petenense")| 
        (ds.field("Taxa")=="Morone saxatilis") |
        (ds.field("Taxa")== "Spirinchus thaleichthys"))

fishf = deltafish.to_table(filter = expr,
                           columns =['SampleID', 'Length', 'Count', 'Taxa'])

In [18]:
fishf.num_rows

1817734

In [19]:
survey = ds.dataset("/home/shares/deltafish/survey",
                    format="parquet",
                    partitioning='hive')

In [20]:
survey.schema

Station: string
Latitude: float
Longitude: float
Date: date32[day]
Datetime: timestamp[ms, tz=America/Los_Angeles]
Survey: int64
Depth: float
SampleID: string
Method: string
Tide: string
Sal_surf: float
Sal_bot: float
Temp_surf: float
Secchi: float
Secchi_estimated: bool
Tow_duration: float
Tow_area: float
Tow_volume: float
Cable_length: float
Tow_direction: string
Notes_tow: string
Notes_flowmeter: string
Source: string

In [21]:
survey_s = survey.to_table(columns=['SampleID','Datetime', 'Station', 'Longitude', 'Latitude'])

In [22]:
fish_j = fishf.join(survey_s, "SampleID").to_pandas()
fish_j.head()

,SampleID,Length,Count,Taxa,Datetime,Station,Longitude,Latitude
0,20mm 803,NaN,0.0,Dorosoma petenense,1995-08-04 10:37:00-07:00,323,-122.286308,38.042889
1,20mm 8030,NaN,0.0,Dorosoma petenense,2003-04-08 11:31:00-07:00,707,-121.707863,38.114693
2,20mm 8031,NaN,0.0,Dorosoma petenense,2003-04-08 11:44:00-07:00,707,-121.707863,38.114693
3,20mm 8032,NaN,0.0,Dorosoma petenense,2003-04-08 11:57:00-07:00,707,-121.707863,38.114693
4,20mm 8033,NaN,0.0,Dorosoma petenense,2003-04-08 12:40:00-07:00,711,-121.662247,38.177418
